# Homework - Deployment

In [1]:
import pandas as pd
import numpy as np
import pickle
import requests
from urllib.request import urlopen
from time import sleep

## Getting the data

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv")

In [3]:
data.head()

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,yes,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12
1,yes,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13
2,yes,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5
3,yes,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7
4,yes,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5


In [4]:
data.describe()

,reports,age,income,share,expenditure,dependents,months,majorcards,active
count,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000
mean,0.456406,33.213103,3.365376,0.068732,185.057071,0.993935,55.267627,0.817286,6.996967
std,1.345267,10.142783,1.693902,0.094656,272.218917,1.247745,66.271746,0.386579,6.305812
min,0.000000,0.166667,0.210000,0.000109,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,25.416670,2.243750,0.002316,4.583333,0.000000,12.000000,1.000000,2.000000
50%,0.000000,31.250000,2.900000,0.038827,101.298300,1.000000,30.000000,1.000000,6.000000
75%,0.000000,39.416670,4.000000,0.093617,249.035800,2.000000,72.000000,1.000000,11.000000
max,14.000000,83.500000,13.500000,0.906320,3099.505000,6.000000,540.000000,1.000000,46.000000


## Question 1
- Install Pipenv
- What's the version of pipenv you installed?
- Use --version to find out

In [5]:
!pipenv --version

pipenv, version 2022.10.10


## Question 2
- Use Pipenv to install Scikit-Learn version 1.0.2
- What's the first hash for scikit-learn you get in Pipfile.lock?

Answer: `"sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"`

## Getting the models

In [6]:
PREFIX = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework"

## Question 3
- Write a script for loading these models with pickle
- Score this client:

`{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}`

What's the probability that this client will get a credit card?

- 0.162
- 0.391
- 0.601
- 0.993

In [7]:
model = pickle.load(urlopen(PREFIX + "/model1.bin"))
dv = pickle.load(urlopen(PREFIX + "/dv.bin"))

input_data = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
X_input = dv.transform(input_data)

model.predict_proba(X_input)[0, 1]

0.16213414434326598

## Question 4
Now let's serve this model as a web service

- Install Flask and gunicorn (or waitress, if you're on Windows)
- Write Flask code for serving the model
- Now score this client using requests:
```python
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```
What's the probability that this client will get a credit card?

- 0.274
- 0.484
- 0.698
- 0.928

In [8]:
# Assuming the Flask app is running:

url = "http://localhost:5000/predict"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

{'probability': 0.9282218018527452}

## Question 5

Download the base image svizor/zoomcamp-model:3.9.12-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

- 15 Mb
- 125 Mb
- 275 Mb
- 415 Mb

In [9]:
!docker images svizor/zoomcamp-model

REPOSITORY              TAG           IMAGE ID       CREATED      SIZE
svizor/zoomcamp-model   3.9.12-slim   571a6fdc554b   8 days ago   125MB


## Question 6
Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```
What's the probability that this client will get a credit card now?

- 0.289
- 0.502
- 0.769
- 0.972

In [10]:
!docker run -d --rm --name deployment_container -p 5000:5000 deployment:latest

sleep(10) #Assuring that the container is built entirely

url = "http://localhost:5000/predict"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

2bfdf1de6e3f9c5910ff7a7939cdcecac23228ae2d6690d39e1607340da4a12f


{'probability': 0.9282218018527452}

In [11]:
!docker stop deployment_container

deployment_container
